# MFW 2005 + visualización con flopy
## Luis M. de la Cruz
<font color=#FF0000>**Última actualización: mar may 26 18:10:09 CDT 2020**</font>

En este ejemplo se muestra como graficar los resultados de una simulación de MODFLOW 2005 con las herramientas de flopy.

## Preparamos los datos para la visualización

In [ ]:
import numpy as np
import flopy

In [ ]:
# Definimos el nombre de la simulación
modelname = 'ejem_01_MF2005'

# Detalles del ejemplo
Lx = 1000.
Ly = 1000.
# Definimos el número de contornos a graficar
levels = np.linspace(0.,10.,11) 

### Lectura de los datos binarios
Flopy tiene herramientas para realizar gráficos más *ad hoc* para los resultados de MODFLOW. Estas herramientas están en el módulo PlotMapView (antes ModelMap). En el ejemplo siguiente se muestra cómo usar este módulo para graficar las condiciones de frontera (IBOUND), la malla, los contornos de la carga y los vectores de velocidad.

In [ ]:
# Se importa la herramienta para leer los archivos binarios.
import flopy.utils.binaryfile as bf

# Lectura de la salida de MODFLOW
hds = bf.HeadFile(modelname+'.hds')
head = hds.get_data(totim=1.0)

# Alternativamente se puede usar lo siguiente para leer los datos
#times = hds.get_times()
#head = hds.get_data(totim=times[-1])

print(type(hds))
print(type(head))

# Se obtiene la información del archivo con extensión .cbc
cbb = bf.CellBudgetFile(modelname+'.cbc')

print(type(cbb))

# Se obtienen datos del flujo
times = hds.get_times()
frf = cbb.get_data(text='FLOW RIGHT FACE', totim=times[-1])[0]
fff = cbb.get_data(text='FLOW FRONT FACE', totim=times[-1])[0]

print(type(frf), type(fff))

print('Tiempos de la simulación: ', times)
print('Dimensiones : head {}, frf {}, fff {}'.format(head.shape,frf.shape, fff.shape))
print('Carga hidráulica:\n', head)
print('frf :\n', frf)
print('fff :\n', fff)

### Cargamos el modelo de MODFLOW (ya ejecutado)

In [ ]:
modelname = 'ejem_01_MF2005'
mf = flopy.modflow.Modflow.load(modelname + '.nam', version='mf2005',)

In [ ]:
# Se define una figura de 10 x 10
#import matplotlib.pyplot as plt
#fig = plt.figure(figsize=(8,6))

# Se crea un mapa del modelo para visualización
modelmap = flopy.plot.PlotMapView(model=mf, layer=0)
print(type(modelmap))

lc = modelmap.plot_grid()   # La malla
qm = modelmap.plot_ibound() # Condiciones de frontera
cs = modelmap.contour_array(head, levels=levels) # Contornos
quiver = modelmap.plot_discharge(frf, fff, head=head) # Vectores


In [ ]:
fig = plt.figure(figsize=(14,6))

# Se agrega una subgráfica y se obtienen sus ejes
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)

modelmap1 = flopy.plot.PlotMapView(model=mf, ax = ax1, layer=0)
qm = modelmap1.plot_ibound()
lc = modelmap1.plot_grid()

modelmap2 = flopy.plot.PlotMapView(model=mf, ax = ax2, layer=0)
lc = modelmap2.plot_grid()
cs = modelmap2.contour_array(head, levels=levels)
quiver = modelmap2.plot_discharge(frf, fff, head=head)

ax3 = fig.add_subplot(133)
modelmap3 = flopy.plot.PlotMapView(model=mf, ax = ax3, layer=0)
modelmap3.plot_array(head, cmap='GnBu')
quiver = modelmap3.plot_discharge(frf, fff, head=head)


### Otras herramientas de visualización

A través del modelo, `mf`, se pueden realizar también varios tipos de gráficos:

In [ ]:
mf.modelgrid.plot();

In [ ]:
hdobj = flopy.utils.HeadFile(modelname+'.hds', model=mf)
hdobj.plot()#contour=True,
#           clabel=True,
#           colorbar=True)
#           filename_base='imagen')

In [ ]:
fig = plt.figure(figsize=(14,4))

# Se agrega una subgráfica y se obtienen sus ejes
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

modelmap1 = flopy.plot.PlotMapView(model=mf, ax = ax1, layer=0)
qm = modelmap1.plot_ibound()
lc = modelmap1.plot_grid()

modelmap2 = flopy.plot.PlotMapView(model=mf, ax = ax2, layer=0)
cs = modelmap2.contour_array(head, levels=levels)

#hdobj.plot(axes = ax3)#, colorbar=True)
modelmap3 = flopy.plot.PlotMapView(model=mf, ax = ax3, layer=0)
#modelmap3.plot_array(head, cmap='GnBu')
quiver = modelmap3.plot_discharge(frf, fff, head=head)

In [ ]:
mf.plot()